In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras

In [2]:
df_in = pd.read_csv('X_values_FBL_SC_aperture_updated.txt')
df_in.columns = ['Q_init','rms_time','rms_laser','Phase_gun','Amplitude_gun','Solenoid_T','Phase_b1','Amplitude_b1','Phase_b2','Amplitude_b2','Phase_b3','Amplitude_b3']
df_in.tail(5)

,Q_init,rms_time,rms_laser,Phase_gun,Amplitude_gun,Solenoid_T,Phase_b1,Amplitude_b1,Phase_b2,Amplitude_b2,Phase_b3,Amplitude_b3
153994,0.002273,0.003799,0.5,-7.342794,20.0,0.050166,79.402464,2.154804,73.132979,6.564727,-25.964305,8.478428
153995,0.001119,0.008010,0.5,-9.611701,20.0,0.059934,103.096533,6.884488,-29.237887,4.313310,-13.774343,3.827870
153996,0.002196,0.006606,0.5,-4.035026,20.0,0.070726,-36.090227,8.171303,119.647934,9.551180,-144.992903,6.662927
153997,0.001570,0.006417,0.5,4.726403,20.0,0.053106,139.381285,9.769977,61.960262,6.175585,112.500376,2.036236
153998,0.003208,0.002119,0.5,-8.265603,20.0,0.060178,97.602394,8.427523,128.854482,1.037971,150.529381,9.366261


In [3]:
df_out = pd.read_csv('Y_values_FBL_SC_aperture_updated.txt',delimiter=',')
df_out.columns = ['Emittance_z','Bunch_length', 'Kinetic_energy', 'Energy_deviation','TOF','Trans_size','Trans_emittance','Q']
df_out.tail(5)

,Emittance_z,Bunch_length,Kinetic_energy,Energy_deviation,TOF,Trans_size,Trans_emittance,Q
153994,0.70925,0.93255,2.73240,4.9175,26.176,0.16786,0.045786,-0.000010
153995,4.46190,1.27580,2.25960,4.1974,26.249,0.25654,0.040952,-0.000021
153996,4.05510,0.45535,1.22440,9.1261,26.454,2.26010,0.250090,-0.001248
153997,5711.60000,89.71800,0.99075,223.7300,26.939,0.83326,0.105850,-0.000011
153998,59.49100,34.35700,0.50739,36.0250,27.540,0.55022,0.035567,-0.000025


In [5]:
#FILTERING

df_in = df_in[df_out['Kinetic_energy'] > 1.0]
df_out = df_out[df_out['Kinetic_energy'] > 1.0]

df_in.replace([np.inf], np.nan, inplace=True)
df_out.replace([np.inf], np.nan, inplace=True)

df_in = df_in[df_out['Kinetic_energy'].notna()]
df_out = df_out[df_out['Kinetic_energy'].notna()]


#-----------------------------------------------

df_in = df_in[df_out['Q'] > -200.0e-6]
df_out = df_out[df_out['Q'] > -200.0e-6]

df_in = df_in[df_out['Q'] < -20.0e-6]
df_out = df_out[df_out['Q'] < -20.0e-6]



55724

In [11]:
#BUNCH LENGTH TO BUNCH DURATION TRANSFORMATION

length = np.asarray(df_out['Bunch_length'].tolist()) #in mm
k_energy = np.asarray(df_out['Kinetic_energy'].tolist()) #in MeV 
gamma = (k_energy/0.511+1)
beta = np.sqrt(gamma**2-1)/gamma
bunch_duration = length*1e-3/beta/3e8
df_out['Bunch_duration'] = bunch_duration

In [12]:
#CALCULATE TIME OF FLIGHT JITTER USING FUNCTION
from ToF_Jitter_callable import ToF_Jitter

ToF_class = ToF_Jitter('Results_inverted_0323.txt')
jitter_values = []
energy_jitters = []
for i in range(len(df_out['Bunch_length'])):
    try:
        jitters = ToF_class.Jitter_calculator(df_in['Phase_gun'].iloc[i],df_in['Amplitude_gun'].iloc[i],
                                            df_in['Phase_b1'].iloc[i],df_in['Amplitude_b1'].iloc[i],
                                            df_in['Phase_b2'].iloc[i],df_in['Amplitude_b2'].iloc[i],
                                            df_in['Phase_b3'].iloc[i],df_in['Amplitude_b3'].iloc[i])
    except:
        jitters = [[np.inf],[np.inf]]
        
    jitter_values.append(jitters[0][0])
    energy_jitters.append(jitters[1][0])
    if (i%100)==0:
        print(i)
jitter_values = np.asarray(jitter_values)
energy_jitters = np.asarray(energy_jitters)

#print(jitter_values)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200


/home/ba/PhD_Thesis/scripts/Surrogated_Model_UED/ToF_Jitter_callable.py:193: RuntimeWarning: invalid value encountered in sqrt
  t3 = L3/(c_light*np.sqrt(1.0-1.0/(gamma_2**2))) #time of flight needed to cross L3
/home/ba/PhD_Thesis/scripts/Surrogated_Model_UED/ToF_Jitter_callable.py:195: RuntimeWarning: invalid value encountered in power
  L3_factor = L3/(me_SI*c_light**3*(gamma_2**2-1)**(3.0/2.0))


1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [13]:
#FINALLY, CREATE THE TIME RESOLUTION COLUMN AND SAVE 
df_out['ToF_Jitter'] = jitter_values
df_out['E_Jitter'] = energy_jitters
df_out['Time_Resolution'] = np.sqrt(bunch_duration**2+jitter_values**2)

with open('Out.txt','w') as file_out:
    df_out.to_csv(file_out)
with open('In.txt','w') as file_in:
    df_in.to_csv(file_in)

    
#print(df_out['Time_Resolution'].min())
#print(df_in.loc[53585])
#print(df_out.loc[53585])

#print(df_out['Bunch_duration'].iloc[37676])
#print(df_out['ToF_Jitter'].iloc[37676])

2.4877790108867664e-13
